In [0]:
# import delta
# def table_exists(catalog, database, table):
#     count = (
#         spark.sql(f"SHOW TABLES FROM {catalog}.{database}")
#         .filter(f"database = '{database}' AND tableName = '{table}'")
#         .count()
#     )
#     return count == 1

In [0]:
# Outra abordagem para a função table_exists utilizzando SHOW TABLES IN
# import delta
# def table_exists(catalog, database, table):
#     df_tables = spark.sql(f"SHOW TABLES IN {catalog}.{database}")
#     # Usando aspas no filtro para evitar erro de parsing
#     count = (
#         df_tables
#         .filter(f"database = '{database}' AND tableName = '{table}'")
#         .count()
#     )
#     return count > 0

# Vantagem: não precisa tentar acessar a tabela; funciona rápido se o catálogo já está populado.
# Observação: se o database na listagem não vier preenchido (isso acontece dependendo da versão/configuração), o filtro pode falhar.


In [0]:
# Outra abordagem para a função table_exists usando DESCRIBE TABLE com Try/Except (mais robusto)
import delta
def table_exists(catalog, database, table):
    try:
        spark.sql(f"DESCRIBE TABLE {catalog}.{database}.{table}")
        return True
    except Exception:
        return False

# Vantagem: evita depender do formato do SHOW TABLES e do preenchimento da coluna database.
# Funciona mesmo se a tabela for externa, view ou Delta table no Unity Catalog.

In [0]:
catalog = 'bronze'
schema = 'upsell'
tablename = dbutils.widgets.get('tablename')
id_field = dbutils.widgets.get('id_field')
timestamp_field = dbutils.widgets.get('timestamp_field')

In [0]:
if not table_exists(catalog, schema, tablename):

    print('Tabela não existe, criando...')

    df_full = spark.read.format('parquet').load(f'/Volumes/raw/full_load/{tablename}/')
    
    (df_full.coalesce(1)
    .write.format("delta")
    .mode("overwrite")
    .saveAsTable(
    f"{catalog}.{schema}.{tablename}")
    )

else:
    print('Tabela já existe, ignorando full-load')

In [0]:
spark.read \
    .format("parquet") \
    .load(f"/Volumes/raw/data/cdc/{tablename}/") \
    .createOrReplaceTempView(f"view_{tablename}")

query = f"""
    select *
    from view_{tablename}
    qualify row_number() over (partition by {id_field} order by {timestamp_field} desc) = 1
"""

df_cdc_unique = spark.sql(query)

In [0]:
bronze = delta.DeltaTable.forName(spark, f'{catalog}.{schema}.{tablename}')

bronze.alias('b') \
        .merge(df_cdc_unique.alias('d'), f'b.{id_field} = d.{id_field}') \
        .whenMatchedDelete(condition="d.op = 'D'") \
        .whenMatchedUpdateAll(condition="d.op = 'U'") \
        .whenNotMatchedInsertAll(condition="d.op = 'I' OR d.op = 'U'") \
        .execute()
